### 任务一

In [ ]:
import torch
import numpy as np


class Net(torch.nn.Module):
    def forward(self, x):
        # 第一层神经网络
        z1 = 2 * x[0] + x[1]
        z2 = x[0] * 3 * x[2]
        z3 = -x[2]
        # 第二层神经网络
        u1 = u1 = torch.sin(z1)
        u2 = 2 * x[2] + z2
        u3 = 2 * z1 + z3
        # 第三层神经网络
        v1 = u1-u3
        v2 = torch.sin(-u2)
        v3 = u1 * u3
        # 计算y1/y2
        y1 = v1 ** 2 + v2 ** 2
        y2 = v2 * v3
        return y1, y2


x = torch.tensor([1, 2, 0],dtype=torch.float, requires_grad=True)
model = Net()

y1, y2 = model(x)

# 分别对 y1 和 y2 执行反向传播
y1.backward(retain_graph=True)
print(x.grad)
x.grad.zero_()  # 清空梯度
y2.backward()

# 查看输入 x 的梯度
print(x.grad)

###

In [22]:
import numpy as np

class Computational_Graph:
    def __init__(self) -> None:
        self.x1 = 0
        self.x2 = 0
        self.x3 = 0
         # 局部函数
        self.z1=0
        self.z2=0
        self.z3=0
        self.z4=0 #  加入虚拟节点

        self.u1=0
        self.u2=0
        self.u3=0

        self.v1=0
        self.v2=0
        self.v3=0

        self.y1=0
        self.y2=0

    def forward(self, x1, x2, x3):
        self.x1 = x1
        self.x2 = x2
        self.x3 = x3
        # 在前向传播函数中使用权重和偏差进行计算
        self.z1 = 2*self.x1 + self.x2
        self.z2 = self.x1 * self.x3*3
        self.z3 = -self.x3
        self.z4 = self.x3    # 加入虚拟节点z4

        self.u1 = np.sin(self.z1)
        self.u2 = 2 * self.z4 + self.z2
        self.u3 = 2 * self.z1 + self.z3

        self.v1 = self.u1 - self.u3
        self.v2 = np.sin(-self.u2)
        self.v3 = self.u1 * self.u3

        y1 = self.v1 ** 2 + self.v2 ** 3
        y2 = self.v2 * self.v3

        return  y1, y2


    def backward(self):
        # # 在反向传播函数中完成梯度的计算
        # # 根据链式法则计算各个梯度
        dy1_dv1 = 2 * self.v1
        dy1_dv2 = 3 * (self.v2 ** 2)
        dy2_dv2 = self.v3
        dy2_dv3 = self.v2

        # # 通过链式法则计算最终的梯度
        dv_du = np.array([
            [1, 0, -1],       # dv1/du1, dv1/du2, dv1/du3
            [0, -np.cos(-self.u2), 0],  # dv2/du1, dv2/du2, dv2/du3
            [self.u3, 0, self.u1]     # dv3/du1, dv3/du2, dv3/du3
        ])

        # Gradient matrix for u to z
        du_dz = np.array([
            [np.cos(self.z1), 0, 0, 0],   # du1/dz1, du1/dz2, du1/dz3, du1/dz4
            [0, 1, 0, 2],             # du2/dz1, du2/dz2, du2/dz3, du2/dz4
            [2, 0, 1, 0]              # du3/dz1, du3/dz2, du3/dz3,du3/dz4
        ])

        # Gradient matrix for z to x
        dz_dx = np.array([
            [2, 1, 0],            # dz1/dx1, dz1/dx2, dz1/dx3
            [3 * self.x3, 0, 3 * self.x1],  # dz2/dx1, dz2/dx2, dz2/dx3
            [0, 0, -1],           # dz3/dx1, dz3/dx2, dz3/dx3
            [0, 0, 1]             # dz4/dx1, dz4/dx2, dz4/dx3
        ])

        dy1_dx = dz_dx.T @ du_dz.T @ dv_du.T @ [[dy1_dv1], [dy1_dv2], [0]]
        dy2_dx = dz_dx.T @ du_dz.T @ dv_du.T @ [[0], [dy2_dv2], [dy2_dv3]]

        return dy1_dx, dy2_dx


In [23]:
# 使用示例
graph = Computational_Graph()
y1, y2= graph.forward(2, 1, 3)
print(y1, y2)
gradients = graph.backward()
print("Gradients:", gradients)

64.08711522201764 -6.078667517564844
Gradients: (array([[ 45.24866101],
       [ 27.32040539],
       [-24.26642612]]), array([[25.74837188],
       [ 0.06138621],
       [23.6466917 ]]))


In [24]:
graph = Computational_Graph()
y1, y2= graph.forward(1, 1, 1)
print(y1, y2)
gradients = graph.backward()
print("Gradients:", gradients)

24.490479942112778 0.6766170068463221
Gradients: (array([[ 55.76452115],
       [ 29.05602944],
       [-13.63032285]]), array([[-9.55244091],
       [-4.47599238],
       [-1.13608365]]))
